In [4]:
import os
import cv2
import PIL
import sys
import csv
import copy
import scipy
import base64
import cv2 as cv 
import scipy.misc
import numpy as np
from PIL import Image
import scipy.io as sio
from pandas import read_csv
from keras.utils import np_utils
from keras.models import Model, Sequential, model_from_json  
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation

In [7]:
class OpenCv:
    @staticmethod
    def countOfImage(path):
        dirs = os.listdir(path)
        i = 0
        for f in dirs:
            i += 1
        print(i)
        return i
    
    @staticmethod
    def learn (self, pathToDatas, pathToCascade):
        
        countPos = self.countOfImage(pathToDatas + '\\positive')
        countNeg = self.countOfImage(pathToDatas + '\\negative')
        if os.path.exists(pathToCascade):
            print("Directory to createdCascade already exists!")
            #return
        #os.makedirs(pathToCascade)
        os.system(r'opencv_createsamples -info ' + pathToDatas + '\\positive.dat -vec ' + 
                  pathToDatas +'\\samples.vec -w 30 -h 30')

        os.system(r'opencv_traincascade -data ' + pathToCascade + ' -vec ' + pathToDatas +
                  '\\samples.vec -bg ' + pathToDatas + 
                  '\\negative.dat -numStages 16 -minHitRate 0.999 -maxFalseAlarmRate 0.5 -numPos ' + 
                  #'15' + ' -numNeg ' + '24' + 
                  str(0.8*countPos) + ' -numNeg ' + str(0.8*countNeg) + 
                  ' -w 30 -h 30 -mode ALL -precalcValBufSize 1024 -precalcIdxBufSize 1024')
        
     
    #str(countPos) + ' -numNeg ' + str(countNeg) + 
    ##numPos and numNeg should not be static!
    @staticmethod
    def detectFaceOnImage(nameOfImage):
        cascade = cv.CascadeClassifier( "Haar3/cascade.xml") 
        #img = cv.imread(nameOfImage, 1)
        img = nameOfImage
        crop_img = copy.deepcopy(img)
        rects = cascade.detectMultiScale(img, scaleFactor=1.01, minNeighbors = 4, minSize= (40, 40)) 
        #Delete this block in future
        #if (rects != () ):  
        if (len(rects) != 0 ):
            coords = rects[rects[:,2].argmax(axis=0),:] # string with max elem in 3rd coloumn
        else:
            print ("No face was detected")
            return None, None 
        
        for x, y, w, h in rects: 
            img = cv.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 2) 
        
        cor = np.array([coords[0],coords[1],coords[0]+coords[2],coords[1]+coords[3]])
        crop_img = crop_img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        rez_img = PIL.Image.fromarray(crop_img)
        rez_img = rez_img.resize((96,96), PIL.Image.ANTIALIAS)
        return cor, rez_img
    
    @staticmethod
    def defaultHaarClassificator(nameOfImage):
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        #img = cv.imread(nameOfImage, 1)
        img = nameOfImage
        crop_img = copy.deepcopy(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = face_cascade.detectMultiScale(gray, 1.3, 5)
        if (len(rects) != 0 ): 
            coords = rects[rects[:,2].argmax(axis=0),:] # string with max elem in 3rd coloumn
        else:
            print ("No face was detected")
            return None, None
        
        cor = np.array([coords[0], coords[1], coords[0]+coords[2], coords[1]+coords[3]])
        crop_img = crop_img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        rez_img = PIL.Image.fromarray(crop_img)
        rez_img = rez_img.resize((96,96), PIL.Image.ANTIALIAS)
        return cor, rez_img


In [8]:
o = OpenCv()
o.learn(o, 'datas','CompTechCascade')

11543
94702
Directory to createdCascade already exists!


In [3]:
def CNN(image):
    def create_model():
        height, width, deph = 96,96,1
        batch_size = 512
        num_epochs = 5
        hidden_size = 512 
        num_classes = 2 
        conv_depth_1 = 24
        pool_size = (2, 2)
        kernel_size = 2
        drop_prob_1 = 0.25
        drop_prob_2 = 0.5

        inp = Input(shape=(1, height, width))
        conv_1 = Convolution2D(24, 2, 2, border_mode='same', activation='relu')(inp)
        drop_1 = Dropout(drop_prob_1)(conv_1)
        flat_1=Flatten()(drop_1)

        hidden_1 = Dense(batch_size, activation='relu')(flat_1)
        drop_2 = Dropout(drop_prob_2)(hidden_1)
        out = Dense(num_classes, activation='softmax')(drop_2)

        model = Model(input=inp, output=out)

        model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model

    def read_jpeg_training_data():
        x_train = []
        y_train = []
        length = 0
        dirpath = 'datas/positive'
        img_ext = '.png' 
        img_names = [ os.path.join(dirpath,x) for x in os.listdir( dirpath ) if x.endswith(img_ext) ]
        length += len(img_names)
        for i in img_names:
            x_train.append(read_jpeg(i))
            y_train.append(1)  
        dirpath = 'datas/negative'
        img_names = [ os.path.join(dirpath,x) for x in os.listdir( dirpath ) if x.endswith(img_ext) ]
        for i in img_names:
            x_train.append(read_jpeg(i))
            y_train.append(0) 
        length += len(img_names)
        return x_train, y_train, length   

    def train_model(model):
        batch_size = 512
        num_epochs = 5
        height, width, deph = 96,96,1
        x_train, y_train, set_size = read_jpeg_training_data()
        y_train = np_utils.to_categorical(y_train, 2) 
        x_train = np.array(x_train)
        x_train = x_train.astype('float32')
        x_train /= 225
        x_train = x_train.reshape(set_size,1,height,width)
        model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs, verbose=0,validation_split=0.05)
        return model

    def evaluate_numpy_array(img):
        assert isinstance(img, np.ndarray)
        model = load_model()
        img = img.reshape(1,1,96,96)
        predictions = model.predict(img, batch_size=32, verbose=0)
        
        return bool(int(round(predictions[0][1])))

    def read_jpeg(path):
        im = Image.open(path).convert('L')
        X = list(im.getdata())
        X = np.array(X)
        return X

    def converet_PIL_img(img):
        img = img.convert('L')
        img = list(img.getdata())
        return (np.array(img))

    def load_model():
        json_file = open('network.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights("network.csv")
        return model

    def save_model(model):
        model_json = model.to_json()
        with open("network.json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights("network.csv")
    
    
    return evaluate_numpy_array(converet_PIL_img(image))
    

In [4]:
def faceDetector(base64Img):
    
    imgdata = base64.b64decode(base64Img)
    filename = 'encode_image.jpg'  
    with open(filename, 'wb') as f:
        f.write(imgdata)

    image = cv.imread('encode_image.jpg', 1)    
    
    o = OpenCv()
    #x, img = o.detectFaceOnImage(image)
    x, img = o.defaultHaarClassificator(image)
    #o.learn(o, 'datas','CompTech')

    if (img):
        if (CNN(img)):
            return (x)
        else:
            return False 
    else:
        return False

In [5]:
import base64
test_im = open("Artem.jpg", 'rb').read() 
enc_base64_img = base64.b64encode(test_im)
faceDetector(enc_base64_img)
print(faceDetector(enc_base64_img))



[ 49 245 342 538]


In [7]:
o = OpenCv()
o.learn(o, 'datas','CompTechCascade')
#x, img = o.defaultHaarClassificator('Artem.jpg')
#%matplotlib inline
#from matplotlib import pyplot as plt
#plt.imshow(img)
#plt.show()


11543
54791
